In [1]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np

import os
import time
from tqdm import tqdm

In [ ]:
class nermodel(torch.nn.Module):
    def __init__(self, params):
        super(nermodel, self).__init__()
        self.device = params.device
        self.hidden_layer = params.hl
        self.num_layer = params.nl
        self.bidir = params.bidir
        self.inpsize = params.inpsize 
        # keep batch first
        self.rnn = nn.LSTM()
        